<a href="https://colab.research.google.com/github/Polarbeargo/PySyft/blob/master/examples/tutorials/Part%207%20-%20Federated%20Learning%20with%20Federated%20Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 7 - Federated Learning with FederatedDataset

Here we introduce a new tool for using federated datasets. We have created a `FederatedDataset` class which is intended to be used like the PyTorch Dataset class, and is given to a federated data loader `FederatedDataLoader` which will iterate on it in a federated fashion.


Authors:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)
- Théo Ryffel - GitHub: [@LaRiffle](https://github.com/LaRiffle)

In [1]:
!pip install syft

     |████████████████████████████████| 184kB 4.8MB/s 
     |████████████████████████████████| 1.4MB 49.2MB/s 
     |████████████████████████████████| 204kB 53.9MB/s 
     |████████████████████████████████| 389kB 42.0MB/s 
     |████████████████████████████████| 71kB 27.8MB/s 
     |████████████████████████████████| 450kB 50.0MB/s 
     |████████████████████████████████| 51kB 21.6MB/s 
     |████████████████████████████████| 276kB 51.0MB/s 
     |████████████████████████████████| 122kB 49.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/ad/9a/f4/3105b5209674ac77fcca7fede95184c62a95df0196888e0e76
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


We use the sandbox that we discovered last lesson

In [2]:
import torch as th
import syft as sy
sy.create_sandbox(globals(), verbose=False)

Setting up Sandbox...
Done!


Then search for a dataset

In [0]:
boston_data = grid.search("#boston", "#data", verbose=False, return_counter=False)
boston_target = grid.search("#boston", "#target", verbose=False, return_counter=False)

We load a model and an optimizer

In [0]:
n_features = boston_data['alice'][0].shape[1]
n_targets = 1
model = th.nn.Linear(n_features, n_targets)
optimizer = th.optim.SGD(params=model.parameters(),lr=0.0000001)

Here we cast the data fetched in a `FederatedDataset`. See the workers which hold part of the data.

In [5]:
# Cast the result in BaseDatasets
datasets = []
for worker in boston_data.keys():
    dataset = sy.BaseDataset(boston_data[worker][0], boston_target[worker][0])
    datasets.append(dataset)

# Build the FederatedDataset object
dataset = sy.FederatedDataset(datasets)
print(dataset.workers)

['bob', 'theo', 'jason', 'alice', 'andy', 'jon']


We put it in a `FederatedDataLoader` and specify options

In [0]:
train_loader = sy.FederatedDataLoader(dataset, batch_size=4, shuffle=False, drop_last=False)

And finally we iterate over epochs. You can see how similar this is compared to pure and local PyTorch training!

In [7]:
epochs = 10
for epoch in range(1, epochs + 1):
    loss_accum = 0
    for batch_idx, (data, target) in enumerate(train_loader):

        model.send(data.location)
        
        optimizer.zero_grad()
        pred = model(data)
        loss = ((pred - target)**2).sum()
        loss.backward()
        optimizer.step()
        
        model.get()
        loss = loss.get()
        
        loss_accum += float(loss)
        
        if batch_idx % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * data.shape[0], len(train_loader),
                       100. * batch_idx / len(train_loader), loss.item()))
            
            
    print('Total loss', loss_accum)


Train Epoch: 1 [0/127 (0%)]	Loss: 301401.562500
Train Epoch: 1 [80/127 (16%)]	Loss: 2114.468018
Train Epoch: 1 [160/127 (31%)]	Loss: 10344.879883
Train Epoch: 1 [240/127 (47%)]	Loss: 991.852783
Train Epoch: 1 [320/127 (63%)]	Loss: 210.898224
Train Epoch: 1 [400/127 (79%)]	Loss: 10101.279297
Train Epoch: 1 [480/127 (94%)]	Loss: 4210.054688
Total loss 741109.0919494629
Train Epoch: 2 [0/127 (0%)]	Loss: 2385.207275
Train Epoch: 2 [80/127 (16%)]	Loss: 324.100708
Train Epoch: 2 [160/127 (31%)]	Loss: 7908.136230
Train Epoch: 2 [240/127 (47%)]	Loss: 1015.451782
Train Epoch: 2 [320/127 (63%)]	Loss: 144.818268
Train Epoch: 2 [400/127 (79%)]	Loss: 5754.004395
Train Epoch: 2 [480/127 (94%)]	Loss: 5770.668457
Total loss 275168.7668952942
Train Epoch: 3 [0/127 (0%)]	Loss: 2143.198242
Train Epoch: 3 [80/127 (16%)]	Loss: 216.586853
Train Epoch: 3 [160/127 (31%)]	Loss: 7541.060547
Train Epoch: 3 [240/127 (47%)]	Loss: 992.328613
Train Epoch: 3 [320/127 (63%)]	Loss: 132.603836
Train Epoch: 3 [400/127 (7

# Congratulations!!! - Time to Join the Community!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!

### Star PySyft on GitHub

The easiest way to help our community is just by starring the Repos! This helps raise awareness of the cool tools we're building.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Join our Slack!

The best way to keep up to date on the latest advancements is to join our community! You can do so by filling out the form at [http://slack.openmined.org](http://slack.openmined.org)

### Join a Code Project!

The best way to contribute to our community is to become a code contributor! At any time you can go to PySyft GitHub Issues page and filter for "Projects". This will show you all the top level Tickets giving an overview of what projects you can join! If you don't want to join a project, but you would like to do a bit of coding, you can also look for more "one off" mini-projects by searching for GitHub issues marked "good first issue".

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)